In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# ------------------------------
# Dataset
# ------------------------------
class HandSignDataset(Dataset):
    def __init__(self, data_dir="data/raw"):
        self.samples = []
        self.labels = []
        self.label_map = {chr(i+65): i for i in range(26)}  # A=0 ... Z=25

        # Make sure data directory exists
        os.makedirs(data_dir, exist_ok=True)

        for letter, idx in self.label_map.items():
            path = os.path.join(data_dir, f"{letter}.npy")
            if os.path.exists(path):
                data = np.load(path)
                self.samples.extend(data)
                self.labels.extend([idx] * len(data))

        if len(self.samples) == 0:
            raise RuntimeError(f"No training data found in {data_dir}. "
                               f"Run collect_data.py first!")

        self.samples = torch.tensor(np.array(self.samples), dtype=torch.float32)
        self.labels = torch.tensor(np.array(self.labels), dtype=torch.long)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]


# ------------------------------
# Model
# ------------------------------
class HandSignModel(nn.Module):
    def __init__(self, input_size=63, num_classes=26):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)


# ------------------------------
# Training
# ------------------------------
def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    dataset = HandSignDataset()
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = HandSignModel().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    epochs = 25
    for epoch in range(epochs):
        total_loss = 0
        for X, y in loader:
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}")

    # ✅ make sure models folder exists
    os.makedirs("models", exist_ok=True)
    torch.save(model.state_dict(), "models/best_model.pt")
    print("✅ Model saved at models/best_model.pt")


if __name__ == "__main__":
    main()


Using device: cpu
Epoch 1/25, Loss: 1.9839
Epoch 2/25, Loss: 0.7986
Epoch 3/25, Loss: 0.3966
Epoch 4/25, Loss: 0.1911
Epoch 5/25, Loss: 0.1023
Epoch 6/25, Loss: 0.0673
Epoch 7/25, Loss: 0.0491
Epoch 8/25, Loss: 0.0400
Epoch 9/25, Loss: 0.0322
Epoch 10/25, Loss: 0.0277
Epoch 11/25, Loss: 0.0236
Epoch 12/25, Loss: 0.0213
Epoch 13/25, Loss: 0.0186
Epoch 14/25, Loss: 0.0176
Epoch 15/25, Loss: 0.0159
Epoch 16/25, Loss: 0.0131
Epoch 17/25, Loss: 0.0123
Epoch 18/25, Loss: 0.0119
Epoch 19/25, Loss: 0.0105
Epoch 20/25, Loss: 0.0100
Epoch 21/25, Loss: 0.0092
Epoch 22/25, Loss: 0.0087
Epoch 23/25, Loss: 0.0082
Epoch 24/25, Loss: 0.0074
Epoch 25/25, Loss: 0.0077
✅ Model saved at models/best_model.pt
